## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [1]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')

df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [2]:
# View the column names in the data
df.columns


Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')

In [3]:
# Use the describe function to gather some basic statistics
df.describe()


,client_id,order_id,order_week,order_year,unit_price,unit_cost,unit_weight,qty,line_number
count,54639.000000,5.463900e+04,54639.000000,54639.000000,54639.000000,54639.000000,54639.000000,5.463900e+04,54639.000000
mean,54837.869416,5.470190e+06,11.359139,2022.993064,136.267207,99.446073,5.004116,5.702646e+02,2.979667
std,25487.438231,2.599807e+06,7.023499,0.082997,183.873135,133.164267,5.326599,1.879552e+04,2.436320
min,10033.000000,1.000886e+06,1.000000,2022.000000,0.010000,0.010000,0.000000,0.000000e+00,0.000000
25%,33593.000000,3.196372e+06,6.000000,2023.000000,20.800000,14.840000,1.450000,3.200000e+01,1.000000
50%,53305.000000,5.496966e+06,11.000000,2023.000000,68.310000,49.890000,3.240000,6.800000e+01,3.000000
75%,78498.000000,7.733869e+06,17.000000,2023.000000,173.160000,125.570000,6.890000,1.700000e+02,5.000000
max,99984.000000,9.998480e+06,52.000000,2023.000000,1396.230000,846.270000,46.430000,3.958244e+06,9.000000


In [4]:
# Use this space to do any additional research
# and familiarize yourself with the data.
category_counts = df["category"].value_counts()


In [5]:
# What three item categories had the most entries?
categories = df['category'].value_counts().sort_values(ascending=False)
for category in categories.iloc[0:3].index.values:
    print(category)



consumables
furniture
software


In [6]:
# For the category with the most entries,
# which subcategory had the most entries?
df.loc[df["category"] == categories.index[0]]["subcategory"].value_counts().sort_values(ascending=False).index[0]

'bathroom supplies'

In [7]:
# Which five clients had the most entries in the data?
entry_count_per_client = df['client_id'].value_counts().reset_index()
entry_count_per_client.columns = ['client_id', 'entry_count']
count_df = pd.merge(df, entry_count_per_client, on='client_id', how='left')
top5clients = count_df[['client_id', 'first', 'last','email','phone','job',"entry_count"]].drop_duplicates().sort_values("entry_count", ascending=False).head(5)
top5clients

,client_id,first,last,email,phone,job,entry_count
5,33615,Jessica,Reyes,jessica.reyes@grimes.net,5016915923,"Production designer, theatre/television/film",220
216,66037,Angela,Everett,aeverett@rodriguez.com,729-620-6192x875,Osteopath,211
137,46820,Bryan,Myers,bmyers3696@wilson.com,444.233.9146x233,Chief of Staff,209
470,38378,Alexandra,Young,ayoung9158@stevens.com,265-666-3155x118,Dietitian,207
416,24741,Kendra,Garrett,garrettkendra8479@jefferson.com,808.257.2029,Pension scheme manager,207


In [8]:
# Store the client ids of those top 5 clients in a list.
top5_client_ids = list(top5clients['client_id'])
top5_client_ids


[33615, 66037, 46820, 38378, 24741]

In [9]:
# How many total units (the qty column) did the
# client with the most entries order order?
df[df["client_id"] == top5_client_ids[0]]["qty"].sum()


64313

## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [10]:
# Create a column that calculates the 
# subtotal for each line using the unit_price
# and the qty
df["line_subtotal"] = round(df["unit_price"] * df["qty"], 2)
df


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number,line_subtotal
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1,115164.00
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0,523.95
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6,527.28
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3,1056.18
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1,3902.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54634,Lauren,Reese,Radio producer,977.877.5272x11382,late_reese_4081@montoya-chavez.org,29043,9021716,2023-04-26,17,2023,UCA28532-31-0B,software,logistics,83.13,51.60,2.25,33,8,2743.29
54635,Derrick,Moore,Musician,358.661.5483,derrick.moore.2602@pope.info,41908,6290153,2023-02-08,6,2023,EBD29277-23-7U,consumables,kitchen supplies,206.59,175.46,11.70,47,0,9709.73
54636,Monica,Gutierrez,Graphic designer,294.805.9100x339,gutierrezm3195@morris.org,35176,8692622,2023-03-05,9,2023,EBD30093-55-9Z,consumables,bathroom supplies,65.66,57.31,4.16,475,6,31188.50
54637,Wanda,Solomon,Toxicologist,(311)767-4924,solomonwanda5962@ross.org,24485,7592730,2023-01-18,3,2023,XMZ38239-49-3D,furniture,stands,1.48,1.22,18.04,112,0,165.76


In [11]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound
# for orders over 50 pounds and $10 per
# pound for items 50 pounds or under.
df['total_weight'] = df['unit_weight'] * df['qty']
df["shipping_price"] = round(df['total_weight'].apply(lambda x: x * 10 if x <= 50 else x * 7), 2)
df

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number,line_subtotal,total_weight,shipping_price
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,decor,wall art,1096.80,762.71,7.50,105,1,115164.00,787.50,5512.50
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,consumables,pens,24.95,15.09,1.49,21,0,523.95,31.29,312.90
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,software,project management,13.52,7.86,1.68,39,6,527.28,65.52,458.64
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,consumables,pens,36.42,24.85,1.23,29,3,1056.18,35.67,356.70
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,consumables,misc,195.10,108.17,46.43,20,1,3902.00,928.60,6500.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54634,Lauren,Reese,Radio producer,977.877.5272x11382,late_reese_4081@montoya-chavez.org,29043,9021716,2023-04-26,17,2023,...,software,logistics,83.13,51.60,2.25,33,8,2743.29,74.25,519.75
54635,Derrick,Moore,Musician,358.661.5483,derrick.moore.2602@pope.info,41908,6290153,2023-02-08,6,2023,...,consumables,kitchen supplies,206.59,175.46,11.70,47,0,9709.73,549.90,3849.30
54636,Monica,Gutierrez,Graphic designer,294.805.9100x339,gutierrezm3195@morris.org,35176,8692622,2023-03-05,9,2023,...,consumables,bathroom supplies,65.66,57.31,4.16,475,6,31188.50,1976.00,13832.00
54637,Wanda,Solomon,Toxicologist,(311)767-4924,solomonwanda5962@ross.org,24485,7592730,2023-01-18,3,2023,...,furniture,stands,1.48,1.22,18.04,112,0,165.76,2020.48,14143.36


In [12]:
# Create a column for the total price
# using the subtotal and the shipping price
# along with a sales tax of 9.25%
df["total_price"] = df["line_subtotal"] + df["shipping_price"]
df["total_price"] = round(df["total_price"] + (df["total_price"] * 9.25/100),2)
df


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,subcategory,unit_price,unit_cost,unit_weight,qty,line_number,line_subtotal,total_weight,shipping_price,total_price
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,wall art,1096.80,762.71,7.50,105,1,115164.00,787.50,5512.50,131839.08
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,pens,24.95,15.09,1.49,21,0,523.95,31.29,312.90,914.26
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,project management,13.52,7.86,1.68,39,6,527.28,65.52,458.64,1077.12
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,pens,36.42,24.85,1.23,29,3,1056.18,35.67,356.70,1543.57
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,misc,195.10,108.17,46.43,20,1,3902.00,928.60,6500.20,11364.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54634,Lauren,Reese,Radio producer,977.877.5272x11382,late_reese_4081@montoya-chavez.org,29043,9021716,2023-04-26,17,2023,...,logistics,83.13,51.60,2.25,33,8,2743.29,74.25,519.75,3564.87
54635,Derrick,Moore,Musician,358.661.5483,derrick.moore.2602@pope.info,41908,6290153,2023-02-08,6,2023,...,kitchen supplies,206.59,175.46,11.70,47,0,9709.73,549.90,3849.30,14813.24
54636,Monica,Gutierrez,Graphic designer,294.805.9100x339,gutierrezm3195@morris.org,35176,8692622,2023-03-05,9,2023,...,bathroom supplies,65.66,57.31,4.16,475,6,31188.50,1976.00,13832.00,49184.90
54637,Wanda,Solomon,Toxicologist,(311)767-4924,solomonwanda5962@ross.org,24485,7592730,2023-01-18,3,2023,...,stands,1.48,1.22,18.04,112,0,165.76,2020.48,14143.36,15632.71


In [13]:
# Create a column for the cost
# of each line using unit cost, qty, and
# shipping price (assume the shipping cost
# is exactly what is charged to the client).
df["total_cost"] = round((df["qty"] * df["unit_cost"]) + df["shipping_price"], 2)
df


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_price,unit_cost,unit_weight,qty,line_number,line_subtotal,total_weight,shipping_price,total_price,total_cost
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,1096.80,762.71,7.50,105,1,115164.00,787.50,5512.50,131839.08,85597.05
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,24.95,15.09,1.49,21,0,523.95,31.29,312.90,914.26,629.79
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,13.52,7.86,1.68,39,6,527.28,65.52,458.64,1077.12,765.18
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,36.42,24.85,1.23,29,3,1056.18,35.67,356.70,1543.57,1077.35
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,195.10,108.17,46.43,20,1,3902.00,928.60,6500.20,11364.40,8663.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54634,Lauren,Reese,Radio producer,977.877.5272x11382,late_reese_4081@montoya-chavez.org,29043,9021716,2023-04-26,17,2023,...,83.13,51.60,2.25,33,8,2743.29,74.25,519.75,3564.87,2222.55
54635,Derrick,Moore,Musician,358.661.5483,derrick.moore.2602@pope.info,41908,6290153,2023-02-08,6,2023,...,206.59,175.46,11.70,47,0,9709.73,549.90,3849.30,14813.24,12095.92
54636,Monica,Gutierrez,Graphic designer,294.805.9100x339,gutierrezm3195@morris.org,35176,8692622,2023-03-05,9,2023,...,65.66,57.31,4.16,475,6,31188.50,1976.00,13832.00,49184.90,41054.25
54637,Wanda,Solomon,Toxicologist,(311)767-4924,solomonwanda5962@ross.org,24485,7592730,2023-01-18,3,2023,...,1.48,1.22,18.04,112,0,165.76,2020.48,14143.36,15632.71,14280.00


In [14]:
# Create a column for the profit of
# each line using line cost and line price
df["profit"] = round(df["total_price"] - df["total_cost"], 2)
df

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_cost,unit_weight,qty,line_number,line_subtotal,total_weight,shipping_price,total_price,total_cost,profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,762.71,7.50,105,1,115164.00,787.50,5512.50,131839.08,85597.05,46242.03
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,15.09,1.49,21,0,523.95,31.29,312.90,914.26,629.79,284.47
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,7.86,1.68,39,6,527.28,65.52,458.64,1077.12,765.18,311.94
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,24.85,1.23,29,3,1056.18,35.67,356.70,1543.57,1077.35,466.22
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,108.17,46.43,20,1,3902.00,928.60,6500.20,11364.40,8663.60,2700.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54634,Lauren,Reese,Radio producer,977.877.5272x11382,late_reese_4081@montoya-chavez.org,29043,9021716,2023-04-26,17,2023,...,51.60,2.25,33,8,2743.29,74.25,519.75,3564.87,2222.55,1342.32
54635,Derrick,Moore,Musician,358.661.5483,derrick.moore.2602@pope.info,41908,6290153,2023-02-08,6,2023,...,175.46,11.70,47,0,9709.73,549.90,3849.30,14813.24,12095.92,2717.32
54636,Monica,Gutierrez,Graphic designer,294.805.9100x339,gutierrezm3195@morris.org,35176,8692622,2023-03-05,9,2023,...,57.31,4.16,475,6,31188.50,1976.00,13832.00,49184.90,41054.25,8130.65
54637,Wanda,Solomon,Toxicologist,(311)767-4924,solomonwanda5962@ross.org,24485,7592730,2023-01-18,3,2023,...,1.22,18.04,112,0,165.76,2020.48,14143.36,15632.71,14280.00,1352.71


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [15]:
# Check your work using the totals above
#df.groupby("order_id")["total_cost"].sum()
df[df["order_id"].isin([2742071,2173913,6128929])].groupby("order_id")["total_price"].sum().map("${:,.2f}".format)

order_id
2173913    $162,388.71
2742071    $152,811.89
6128929    $923,441.25
Name: total_price, dtype: object

## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [16]:
# How much did each of the top 5 clients by quantity
# spend? Check your work from Part 1 for client ids.
top_client_spend = df[df["client_id"].isin(top5clients["client_id"])].groupby("client_id")["total_price"].sum().map("${:,.2f}".format)
top_client_spend_df = pd.DataFrame({"client_id":top_client_spend.index, "total_spent": top_client_spend.values})
top_client_spend_df

,client_id,total_spent
0,24741,"$82,268,892.02"
1,33615,"$8,377,308.52"
2,38378,"$12,906,550.87"
3,46820,"$9,743,794.36"
4,66037,"$10,259,514.79"


In [17]:
# Create a summary DataFrame showing the totals for the
# for the top 5 clients with the following information:
# total units purchased, total shipping price,
# total revenue, and total profit. Sort by total profit.
top5client_data = df[df["client_id"].isin(top5clients["client_id"])].groupby("client_id")
summary_df = pd.DataFrame({ 
    "total_units_purchased": top5client_data["qty"].sum().map("{:,}".format),
    "total_shipping_price": top5client_data["shipping_price"].sum().map("${:,.2f}".format),
    "total_revenue": top5client_data["total_price"].sum().map("${:,.2f}".format),
    "total_cost": top5client_data["total_cost"].sum().map("${:,.2f}".format),
    "total_profit": top5client_data["profit"].sum().map("${:,.2f}".format),
}).sort_values(by="total_revenue", ascending=False).reset_index()
summary_df


,client_id,total_units_purchased,total_shipping_price,total_revenue,total_cost,total_profit
0,46820,"75,768","$1,601,448.84","$9,743,794.36","$7,007,191.64","$2,736,602.72"
1,24741,"239,862","$5,126,448.37","$82,268,892.02","$45,688,899.71","$36,579,992.31"
2,33615,"64,313","$1,828,984.89","$8,377,308.52","$6,175,313.91","$2,201,994.61"
3,38378,"73,667","$3,429,455.40","$12,906,550.87","$9,634,720.98","$3,271,829.89"
4,66037,"43,018","$1,395,151.85","$10,259,514.79","$7,004,482.98","$3,255,031.81"


In [18]:
# Format the data and rename the columns
# to names suitable for presentation.
# Currency should be in millions of dollars.
summary_df.rename(columns={
    "client_id": "Client ID",
    "total_shipping_price":"Shipping",
    "total_revenue":"Total Revenue",
    "total_units_purchased":"Units",
    "total_cost": "Total Cost",
    "total_profit": "Total Profit"
}, inplace=True)
for x in ["Total Profit","Total Cost","Total Revenue","Shipping"]:
    summary_df[x] = summary_df[x].replace('[\$,]', '', regex=True).astype(float)/1000000
    summary_df[x] = summary_df[x].map('${:,.2f}M'.format)

#summary_df[["Shipping","Total Revenue","Total Cost","Total Profit"]].apply(lambda x: "${:,.2f}M".format(x/1000000))

summary_df


,Client ID,Units,Shipping,Total Revenue,Total Cost,Total Profit
0,46820,"75,768",$1.60M,$9.74M,$7.01M,$2.74M
1,24741,"239,862",$5.13M,$82.27M,$45.69M,$36.58M
2,33615,"64,313",$1.83M,$8.38M,$6.18M,$2.20M
3,38378,"73,667",$3.43M,$12.91M,$9.63M,$3.27M
4,66037,"43,018",$1.40M,$10.26M,$7.00M,$3.26M


In [19]:
# Sort the updated data by "Total Profit" form highest to lowest
summary_df.sort_values(by="Total Profit", ascending=False)

,Client ID,Units,Shipping,Total Revenue,Total Cost,Total Profit
1,24741,"239,862",$5.13M,$82.27M,$45.69M,$36.58M
3,38378,"73,667",$3.43M,$12.91M,$9.63M,$3.27M
4,66037,"43,018",$1.40M,$10.26M,$7.00M,$3.26M
0,46820,"75,768",$1.60M,$9.74M,$7.01M,$2.74M
2,33615,"64,313",$1.83M,$8.38M,$6.18M,$2.20M
